In [1]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [2]:
''' Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! '''

# Connect to db
con = sqlite3.connect('sqlite_db_pythonsqlite.db')
cur = con.cursor()

# the query
query = """
SELECT
    f.name AS facility_name,
    SUM(CASE 
        WHEN b.memid = 0 THEN b.slots * f.guestcost
        ELSE b.slots * f.membercost
    END) AS total_revenue
FROM 
    Bookings AS b
INNER JOIN 
    Facilities AS f ON b.facid = f.facid
GROUP BY 
    f.name
HAVING 
    total_revenue < 1000
ORDER BY 
    total_revenue;
"""

# query the database
cur.execute(query)

rows = cur.fetchall()

# print result
for row in rows:
    facility_name, total_revenue = row
    print(f"Facility Name: {facility_name}, Total Revenue: {total_revenue}")

# close connection
con.close()

Facility Name: Table Tennis, Total Revenue: 180
Facility Name: Snooker Table, Total Revenue: 240
Facility Name: Pool Table, Total Revenue: 270


In [6]:
''' Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order '''

# Connect to db
con = sqlite3.connect('sqlite_db_pythonsqlite.db')
cur = con.cursor()

# the query
query = """
SELECT
    m1.surname AS member_surname,
    m1.firstname AS member_firstname,
    m2.surname AS recommender_surname,
    m2.firstname AS recommender_firstname
FROM 
    Members AS m1
LEFT JOIN
    Members AS m2
    ON m1.recommendedby = m2.memid
ORDER BY
    m1.surname, m1.firstname;
"""

# query the database
cur.execute(query)

rows = cur.fetchall()

# print result
for row in rows:
    member_surname, member_firstname, recommender_surname, recommender_firstname = row
    recommender_name = f"{recommender_surname}, {recommender_firstname}"
    print(f"Member: {member_surname}, {member_firstname} / Recommender: {recommender_name}")

# close connection
con.close()

Member: Bader, Florence / Recommender: Stibbons, Ponder
Member: Baker, Anne / Recommender: Stibbons, Ponder
Member: Baker, Timothy / Recommender: Farrell, Jemima
Member: Boothe, Tim / Recommender: Rownam, Tim
Member: Butters, Gerald / Recommender: Smith, Darren
Member: Coplin, Joan / Recommender: Baker, Timothy
Member: Crumpet, Erica / Recommender: Smith, Tracy
Member: Dare, Nancy / Recommender: Joplette, Janice
Member: Farrell, David / Recommender: None, None
Member: Farrell, Jemima / Recommender: None, None
Member: GUEST, GUEST / Recommender: None, None
Member: Genting, Matthew / Recommender: Butters, Gerald
Member: Hunt, John / Recommender: Purview, Millicent
Member: Jones, David / Recommender: Joplette, Janice
Member: Jones, Douglas / Recommender: Jones, David
Member: Joplette, Janice / Recommender: Smith, Darren
Member: Mackenzie, Anna / Recommender: Smith, Darren
Member: Owen, Charles / Recommender: Smith, Darren
Member: Pinker, David / Recommender: Farrell, Jemima
Member: Purvie

In [7]:
''' Q12: Find the facilities with their usage by member, but not guests '''

# Connect to the SQLite database
con = sqlite3.connect('sqlite_db_pythonsqlite.db')
cur = con.cursor()

# SQL query to find the facilities with their usage by members (excluding guests)
query = """
SELECT
    f.name AS facility_name,
    COUNT(b.bookid) AS member_usage
FROM
    Bookings AS b
INNER JOIN
    Facilities AS f ON b.facid = f.facid
WHERE
    b.memid != 0
GROUP BY
    f.name
ORDER BY
    f.name;
"""

# Execute the query
cur.execute(query)

# Fetch all rows from the result set
rows = cur.fetchall()

# Display the results
for row in rows:
    facility_name, member_usage = row
    print(f"Facility: {facility_name}, Member Usage: {member_usage}")

# Close the database connection
con.close()

Facility: Badminton Court, Member Usage: 344
Facility: Massage Room 1, Member Usage: 421
Facility: Massage Room 2, Member Usage: 27
Facility: Pool Table, Member Usage: 783
Facility: Snooker Table, Member Usage: 421
Facility: Squash Court, Member Usage: 195
Facility: Table Tennis, Member Usage: 385
Facility: Tennis Court 1, Member Usage: 308
Facility: Tennis Court 2, Member Usage: 276


In [11]:
''' Q13: Find the facilities usage by month, but not guests '''

# Connect to the SQLite database
con = sqlite3.connect('sqlite_db_pythonsqlite.db')
cur = con.cursor()

# SQL query to find the facilities usage by month for members (excluding guests)
query = """
SELECT
    f.name AS facility_name,
    strftime('%m', b.starttime) AS month,
    COUNT(b.bookid) AS member_usage
FROM
    Bookings AS b
INNER JOIN
    Facilities AS f 
    ON b.facid = f.facid
WHERE
    b.memid != 0
GROUP BY
    f.name, month
ORDER BY
    f.name, month;
"""

# Execute the query
cur.execute(query)

# Fetch all rows from the result set
rows = cur.fetchall()

# Display the results
for row in rows:
    facility_name, month, member_usage = row
    print(f"Facility: {facility_name}, Month: {month}, Member Usage: {member_usage}")

# Close the database connection
con.close()

Facility: Badminton Court, Month: 07, Member Usage: 51
Facility: Badminton Court, Month: 08, Member Usage: 132
Facility: Badminton Court, Month: 09, Member Usage: 161
Facility: Massage Room 1, Month: 07, Member Usage: 77
Facility: Massage Room 1, Month: 08, Member Usage: 153
Facility: Massage Room 1, Month: 09, Member Usage: 191
Facility: Massage Room 2, Month: 07, Member Usage: 4
Facility: Massage Room 2, Month: 08, Member Usage: 9
Facility: Massage Room 2, Month: 09, Member Usage: 14
Facility: Pool Table, Month: 07, Member Usage: 103
Facility: Pool Table, Month: 08, Member Usage: 272
Facility: Pool Table, Month: 09, Member Usage: 408
Facility: Snooker Table, Month: 07, Member Usage: 68
Facility: Snooker Table, Month: 08, Member Usage: 154
Facility: Snooker Table, Month: 09, Member Usage: 199
Facility: Squash Court, Month: 07, Member Usage: 23
Facility: Squash Court, Month: 08, Member Usage: 85
Facility: Squash Court, Month: 09, Member Usage: 87
Facility: Table Tennis, Month: 07, Memb